# 프로젝트 Movielnes 영화 추천실습
   
     
        
        

``` wget http://files.grouplens.org/datasets/movielens/ml-1m.zip```

## libaryry import

In [47]:
import os
import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## data improt


In [48]:
#rating_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/ratings.dat'
rating_file_path= './data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## data preprocessing

In [49]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)



print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [50]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)


In [51]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [52]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'count']
data = ratings[using_cols]
data.tail(10)

,user_id,movie_id,count
1000198,6040,2021,3
1000199,6040,2022,5
1000200,6040,2028,5
1000201,6040,1080,4
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [53]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
#movie_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/movies.dat'
movie_file_path= './data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.tail(10)

,movie_id,title,genre
3873,3943,Bamboozled (2000),Comedy
3874,3944,Bootmen (2000),Comedy|Drama
3875,3945,Digimon: The Movie (2000),Adventure|Animation|Children's
3876,3946,Get Carter (2000),Action|Drama|Thriller
3877,3947,Get Carter (1971),Thriller
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [54]:
# 유저 이름을 보기 위해 메타 데이터를 읽어옵니다.
#movie_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/movies.dat'
users_file_path= './data/ml-1m/users.dat'
users_cols = ['user_id', 'gender', 'age','occupation','Zip-code' ] 
users = pd.read_csv(users_file_path, sep='::', names=users_cols, engine='python')
users.tail(10)

,user_id,gender,age,occupation,Zip-code
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [55]:
#user_unique = data['user_id'].unique()
movie_names = movies[['movie_id','title']]
movie_names

#movie_title = {val:idx for idx, val in movies['movie_id'].data,movies['title'].data}
# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
#user_to_idx = {v:k for k,v in users.items()}
#movie_to_idx = {v:k for k,v in  enumerate(artist_unique)}

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
3878,3948,Meet the Parents (2000)
3879,3949,Requiem for a Dream (2000)
3880,3950,Tigerland (2000)
3881,3951,Two Family House (2000)


In [56]:
ratings.nunique()

user_id        6039
movie_id       3628
count             3
timestamp    412911
dtype: int64

## data <분석
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [57]:
# 영화 개수
data['movie_id'].nunique()

3628

In [58]:
# 사용자 수
data['user_id'].nunique()

6039

In [59]:
# 가장 인기 있는 영화 30개 인기수
#TODO movie id 와 movies title 매핑
movie_count= data.groupby('movie_id')['user_id'].count()
#movie_count['movie_id'].map(movies.get).dropna()
#print(movies[movies['movie_id'] == 2858]['title'])   movies[movie_count['movie_id'] == movies['movie_id']]['title']
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [60]:
print(movies[movies['movie_id'] == 2858]['title']) 

2789    American Beauty (1999)
Name: title, dtype: object


## data add 
- 내가 선호하는 영화 5가지 골라서 rating 추가


In [61]:
movies[movies["genre"]== "Action"].head(10)
#movies[movies["genre"]== "Drama"].head(30)

,movie_id,title,genre
8,9,Sudden Death (1995),Action
19,20,Money Train (1995),Action
70,71,Fair Game (1995),Action
143,145,Bad Boys (1995),Action
202,204,Under Siege 2: Dark Territory (1995),Action
224,227,Drop Zone (1994),Action
248,251,"Hunted, The (1995)",Action
312,315,"Specialist, The (1994)",Action
380,384,Bad Company (1995),Action
389,393,Street Fighter (1994),Action


In [62]:

movies[movies['title']=='Bad Company (1995)']['movie_id']


380    384
Name: movie_id, dtype: int64

In [63]:
my_favorite = ['Bad Company (1995)' , 
               'Street Fighter (1994)' ,
               'Bad Boys (1995)' ,
               'Sudden Death (1995)' ,
               'Fair Game (1995)']
my_favorite_id= [384, 393, 145, 9, 71 ]

# id --> title조회
for m in my_favorite_id:
    a = movies[movies['movie_id']==m]['title']
    print(np.array(a.tolist())[0])

print('\n')

# title --> id 조회
for m in my_favorite:
    a = movies[movies['title']==m]['movie_id']
    print(np.array(a.tolist())[0])

Bad Company (1995)
Street Fighter (1994)
Bad Boys (1995)
Sudden Death (1995)
Fair Game (1995)


384
393
145
9
71


In [64]:
data

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [65]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [66]:
 


my_playlist = pd.DataFrame({'user_id': [6041]*5, 
                            'movie_id':my_favorite_id, 
                            'count':4})

if not data.isin({'user_id':['6041']})['user_id'].any(): 
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
#data = data.append(my_playlist)     
data.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,384,4
1,6041,393,4
2,6041,145,4
3,6041,9,4
4,6041,71,4


In [67]:
#data[["user_id","movie_id","count"]].to_csv("test2.csv")

In [68]:
data['user_id'].nunique()
# 1명 추가 됨...

6040

In [69]:
data.nunique()

user_id     6040
movie_id    3628
count          3
dtype: int64

In [70]:
#data['user_id'].isnull().values.sum()
#data['movie_id'].isnull().values.sum()
data.isnull().sum()

user_id     0
movie_id    0
count       0
dtype: int64

## csr matrix 

In [71]:
data

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,6041,384,4
1,6041,393,4
2,6041,145,4
3,6041,9,4


In [72]:
#index ####################################################################

#고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화 indexing 하는 코드.
user_to_idx = {v : k for k, v in enumerate(user_unique)}
movie_to_idx = {v : k for k, v in enumerate(movie_unique)}

# 인덱싱 제대로 되었는지 확인
print(user_to_idx[6039])

temp_user_data = data['user_id'].map(user_to_idx.get)#.dropna()

if len(temp_user_data) == len(data):
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing fail')
    
temp_movie_data = data['movie_id'].map(movie_to_idx.get)#.dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing ok!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing fail')

data




6037
user_id column indexing OK!!
movie column indexing ok!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,2256,4
1,6039,2115,4
2,6039,2054,4
3,6039,1941,4


In [73]:
#
for idx in enumerate(movie_to_idx):
    if idx == movie_to_idx.get(idx):
        print (idx)

In [74]:
num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

print(num_user, num_movie)
# print(data2['count'].shape)

# print(type(data))
print(type(num_user))
# print(type(data2.user_id))
# print(type(data2['count']))
csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)), shape= (num_user, num_movie)) 
#csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)), shape= (6050, 3955))
csr_data

6040 3628
<class 'int'>


<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [75]:
data.nunique()

user_id     6040
movie_id    3628
count          3
dtype: int64

## als_model = AlternatingLeastSquares 모델 훈련

In [76]:
# Implicit AlternatingLeastSquares 모델의 선언
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [77]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [78]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 선호도 파악

In [79]:
# 나의 선호도
me_idx = 6039
me_vector  = als_model.user_factors[me_idx]
print(me_vector)


[ 1.20585635e-01  5.14997244e-01  2.25851148e-01 -1.61496401e-01
 -1.47229463e-01 -7.69542977e-02  1.11479245e-01 -4.26025778e-01
  1.76878408e-01  1.08281210e-01 -6.29849061e-02  1.60818025e-01
  1.53010473e-01 -1.77697763e-01 -1.33486882e-01 -1.68807253e-01
 -2.92613298e-01 -2.11006448e-01 -2.64341980e-01 -3.43844146e-01
 -1.55341461e-01  1.86511889e-01  3.90919261e-02  4.32199150e-01
  5.56866527e-02  2.43453786e-01  4.52501953e-01 -1.59643576e-01
  4.24723886e-02 -3.53823245e-01 -1.10200867e-01  1.04373861e-02
  1.64869756e-01 -1.72783509e-01  1.03452921e-01 -2.66586244e-01
 -2.53590912e-01  1.41603798e-01  3.09007168e-01  1.05225578e-01
  1.57498598e-01  1.85501352e-01 -1.84120640e-01 -2.04824179e-01
  9.36736465e-02  2.90430710e-02  2.01871991e-01 -3.98519844e-01
  3.37535255e-02  2.41440744e-03 -2.51490716e-02  1.34863198e-01
  1.82013616e-01  8.12582746e-02  1.40550330e-01 -1.44643664e-01
  2.24438056e-01  3.25293064e-01 -1.36863798e-01 -3.49273011e-02
  3.20116103e-01 -1.35280

In [80]:
#'Fair Game (1995)'  idx 71 선호도   movie_names
fg_idx = 71
fg_vector = als_model.item_factors[fg_idx]
print(fg_vector)

np.dot(me_vector, fg_vector)

[ 1.45606296e-02  4.22562892e-03  1.64642502e-02  1.03443395e-02
  8.96956772e-03  3.04257777e-03  7.79951923e-03  6.96276501e-03
  8.11246037e-03  6.86151022e-03 -2.73770117e-03  9.86464322e-03
  2.56199744e-02  6.46071136e-03  6.83538662e-03 -1.47427814e-02
  1.25819026e-02 -1.32671511e-02  5.90668712e-03 -1.35412607e-02
  7.76082277e-03 -9.84658860e-03  2.52284972e-05  7.93560687e-03
  1.66436471e-02 -5.61388093e-04  1.94147080e-02  5.22034941e-03
  1.54953273e-02  2.81958771e-03 -1.38746027e-03  6.88248966e-03
  8.10229499e-03  7.18870573e-03 -7.34441355e-03  1.31717222e-02
 -1.30732812e-03  1.09219681e-02 -1.19715128e-02  1.77115668e-02
  9.46534623e-04  6.12718007e-03  2.10093334e-02  6.74275635e-03
  3.21957422e-03 -1.61462836e-03  7.67390523e-03  6.11804798e-03
 -1.34944348e-02  1.37108704e-03  7.26970751e-03  9.15840920e-03
  1.93549320e-02 -4.55852645e-03  1.56037938e-02 -1.14084473e-02
 -2.66449340e-03  8.22013337e-03 -7.02526933e-03  2.42750766e-03
  1.92293078e-02  2.00582

0.021955054

In [81]:
#'Street Fighter (1994)' , idx 393 선호도   movie_names
sf_idx = 393
sf_vector = als_model.item_factors[sf_idx]
print(sf_vector)

np.dot(me_vector, sf_vector)


[ 2.78783366e-02 -1.60862431e-02 -2.80532520e-03  6.98850257e-03
  7.38205388e-03  1.70239843e-02  7.88878091e-03  9.15059540e-03
  4.25247243e-03  2.01538065e-03  3.03242635e-03  2.18307041e-02
 -3.84784676e-03 -6.44285604e-03  1.23096714e-02  4.49788244e-03
  1.23434672e-02  1.25470534e-02 -8.30270164e-03  8.82936083e-03
  4.31326102e-04 -6.28334330e-03  1.20048244e-02  1.09246979e-02
  3.92131433e-02  1.17184669e-02  2.27706153e-02  1.31519800e-02
  5.34158084e-04  2.77781906e-03 -1.34188337e-02  4.02430212e-03
 -1.51043665e-02 -8.76199454e-04 -6.02012267e-03  6.49044756e-03
 -3.89086176e-03 -7.91451242e-03 -7.28617981e-03  1.42066192e-03
  9.92914755e-03 -8.80117877e-04 -4.44951141e-03  9.64094326e-03
  7.13952072e-03  6.97814999e-03  1.15437526e-02 -9.10227560e-03
  1.65747218e-02  6.97740540e-03 -7.87773263e-03  1.73636805e-02
 -3.47179454e-03  1.99217759e-02 -4.71184170e-03 -1.43796569e-02
  4.31591226e-03  1.98480859e-02 -2.22084438e-03 -3.02595552e-03
  7.01386854e-03  1.61915

0.0035015587

In [82]:
#'Bad Boys' , idx 393 선호도   movie_names
bb_idx = 145
bb_vector = als_model.item_factors[bb_idx]
print(bb_vector)

np.dot(me_vector, bb_vector)
# my_favorite = ['Bad Company (1995)' , 
#                'Street Fighter (1994)' ,
#                'Bad Boys' ,
#                'Sudden Death (1995)' ,
#                'Fair Game (1995)']
# my_favorite_id= [384, 393, 145, 9, 71 ]

[ 0.01972422  0.01270903  0.01795704 -0.00310239  0.00719362 -0.00974472
  0.03065216  0.02056098  0.02156397 -0.00724528  0.0059642   0.0302564
 -0.00029469  0.01707384  0.0018978  -0.00992931 -0.00139729  0.01502698
 -0.00932424  0.01057915  0.03173926 -0.00142765  0.03092326 -0.0089839
  0.01855205 -0.01166287  0.00602824  0.00223985  0.01386484 -0.01160471
  0.01275183 -0.00501107 -0.01501325 -0.00023553 -0.00044921 -0.00405361
  0.0062056   0.01745011  0.00553727  0.01595993 -0.00943433  0.03948075
  0.01788843  0.00214732  0.01134262  0.00394808 -0.00138876  0.02226844
  0.01123305  0.01910678 -0.01031897  0.00913744  0.03512677  0.01572662
 -0.01205915  0.03568898  0.00291777 -0.01180238 -0.01195669  0.03145553
  0.00775664  0.0154381   0.01242131 -0.0123402  -0.02480444 -0.01853075
  0.00382398  0.01054869  0.01940282  0.01959198  0.00026627  0.00221449
  0.00766649 -0.00305618  0.02735398 -0.01323176 -0.02008408  0.00418158
  0.00932158 -0.0129641  -0.00464665  0.00175191  0.0

-0.012177842

## 유사 영화 추천

In [83]:
#'Fair Game (1995)'  idx 71 선호도   movie_names
fg_idx = 71
similar_movies = als_model.similar_items(fg_idx, N=15)
similar_movies

[(71, 0.117047764),
 (1179, 0.08142685),
 (2896, 0.08107504),
 (66, 0.07706206),
 (1183, 0.07583729),
 (2240, 0.07511372),
 (65, 0.0746294),
 (1643, 0.074370526),
 (2080, 0.07332266),
 (1199, 0.07287997),
 (1496, 0.07238078),
 (771, 0.071514055),
 (801, 0.07150921),
 (2114, 0.0713062),
 (129, 0.070438035)]

In [84]:
# title 매칭
for m in similar_movies:
    print(movies[movies['movie_id']==m[0]][['title','genre']])

               title   genre
70  Fair Game (1995)  Action
                     title                  genre
1163  Grifters, The (1990)  Crime|Drama|Film-Noir
                     title    genre
2827  Alvarez Kelly (1966)  Western
                                        title            genre
65  Lawnmower Man 2: Beyond Cyberspace (1996)  Sci-Fi|Thriller
                            title              genre
1166  English Patient, The (1996)  Drama|Romance|War
                    title  genre
2171  My Bodyguard (1980)  Drama
              title   genre
64  Bio-Dome (1996)  Comedy
                                            title          genre
1597  Mrs. Brown (Her Majesty, Mrs. Brown) (1997)  Drama|Romance
                          title                                        genre
2011  Lady and the Tramp (1955)  Animation|Children's|Comedy|Musical|Romance
              title   genre
1181  Brazil (1985)  Sci-Fi
                     title          genre
1464  Anna Karenina (1997)  Drama|

## 좋아할 만한 영화 추천

In [85]:
#  
me_idx = 6039
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(me_idx, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(2346, 0.11037615),
 (974, 0.10193868),
 (147, 0.10032947),
 (26, 0.10001735),
 (2214, 0.09408437),
 (770, 0.09402874),
 (1972, 0.09147231),
 (2366, 0.09061906),
 (1643, 0.0880799),
 (1967, 0.08754115),
 (1672, 0.08753151),
 (1191, 0.08729647),
 (2353, 0.086113244),
 (2453, 0.08606567),
 (1126, 0.08532235),
 (1423, 0.084845856),
 (1105, 0.084614456),
 (1970, 0.08448736),
 (1233, 0.084127665),
 (2904, 0.08388387)]

In [86]:
## 변환
a = []
# title 
for m in movie_recommended:
    a.append(movies[movies['movie_id']==m[0]]['title'])
    print(movies[movies['movie_id']==m[0]][['title','genre']])

                           title            genre
2277  Stepford Wives, The (1975)  Sci-Fi|Thriller
              title          genre
962  Algiers (1938)  Drama|Romance
                              title  genre
145  Basketball Diaries, The (1995)  Drama
             title  genre
25  Othello (1995)  Drama
                        title     genre
2145  Number Seventeen (1932)  Thriller
                  title  genre
760  Costa Brava (1946)  Drama
                                                  title   genre
1903  Nightmare on Elm Street 5: The Dream Child, A ...  Horror
                 title                    genre
2297  King Kong (1933)  Action|Adventure|Horror
                                            title          genre
1597  Mrs. Brown (Her Majesty, Mrs. Brown) (1997)  Drama|Romance
                 title                         genre
1898  Labyrinth (1986)  Adventure|Children's|Fantasy
                      title  genre
1626  Rainmaker, The (1997)  Drama
                    

## 코멘트....
- 주입한 데이터를 액션영화 5개만을 골라서 입력해서 그런지 유사 영화와 추천영화의 장르가 전부 액션으로 나왔다.